In [1]:
import torch
import os
from torchvision.utils import save_image
from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

from idem_net_celeba import IdemNetCeleba
from idem_net_mnist import IdemNetMnist
from gen_utils import generate_frequency_noise
from data_loader import load_CelebA

In [2]:
run_id = "celeba20241113-154812"
epoch_num = "_final.pth"

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

print(device)
checkpoint_path = f"checkpoints/{run_id}/{epoch_num}"

if "celeba" in run_id:
  model = IdemNetCeleba(3) # IdemNetMnist()
else:
  model = IdemNetMnist()

state_dict = torch.load(checkpoint_path, weights_only=True, map_location=device)
model.load_state_dict(state_dict)

cpu


<All keys matched successfully>

In [3]:
def save_model_outputs_to_images(model, output_dir, batch_size, fft_noise, data_loader, num_generations, device):
    """
    Generate outputs batchwise from a model and save them as images.

    Parameters:
        model: The PyTorch model to generate outputs.
        dataloader: DataLoader to provide input data in batches.
        output_dir: Directory to save the images.
        fft_noise: Whether to use FFT noise for input.
        data_loader: DataLoader to provide input data in batches.
        device: Device to run the model on (e.g., 'cuda' or 'cpu').
    """

    if fft_noise:
        output_dir = "./data/generated_images_fft"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    counter = 0  # Counter for filenames

    progress = tqdm(range(int(num_generations / batch_size)))
    with torch.no_grad():  # Disable gradient computation
        for _ in progress:

            if fft_noise == False:
                noise = torch.randn([batch_size, 3, 64, 64], device=device)
            else:
                # Get a batch of real images
                real_imgs = next(iter(data_loader))
                real_imgs = real_imgs[0:batch_size].to(device)

                noise = generate_frequency_noise(real_imgs)
                
            outputs = model(noise) # Generate outputs from model
            
            # Assuming outputs are images (e.g., RGB normalized tensors)
            for i, output in enumerate(outputs):
                # Optionally normalize or adjust output here
                output_image = to_pil_image(output)  # Convert to PIL image
                
                # Save image to file
                filename = os.path.join(output_dir, f"output_{counter:05d}.png")
                output_image.save(filename)
                
                counter += 1  # Increment counter for unique filenames

    print(f"Images saved to {output_dir}")

# Example Usage
# Assuming you have a DataLoader `dataloader` and a PyTorch model `model`

output_dir = "./data/generated_images"

batch_size = 256

# get the data loader
data_loader, _ = load_CelebA(batch_size=batch_size)

# Save images from model
save_model_outputs_to_images(model.to(device), output_dir, batch_size, num_generations=10, device=device)

FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=0B7EVK8r0v71pZjFTYXZWM3FlRnM

but Gdown can't. Please check connections and permissions.